
# Fertiliser emissions

In [ ]:
import numpy as np
import logging
#logging.getLogger("imperative_model").setLevel(logging.DEBUG)
#logging.basicConfig(level=logging.DEBUG)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import load_model
model, recipe_data = load_model.load_model()

In [ ]:
%run load_model_fertilisers.py
define_fertiliser_model(model)

In [ ]:
flows_sym = model.to_flows(recipe_data)

In [ ]:
func = model.lambdify(list(flows_sym.value.values), recipe_data)

In [ ]:
# Define a demand profile for the fertilisers

In [ ]:
tvals = np.arange(2020, 2051, 1)
#tvals

In [ ]:
t = sy.symbols("t")
#t

In [ ]:
def linear_ramp(t1, t2, y1, y2):
    frac = sy.Min(1, sy.Max(0, (t - t1) / (t2 - t1)))
    return y1 + frac * (y2 - y1)

In [ ]:
#import matplotlib.pyplot as plt
import bqplot.pyplot as plt
def evalt(y):
    return np.vectorize(sy.lambdify(t, y))(tvals)
def plott(y):
    return plt.plot(tvals, evalt(y))
#plt.figure()
#plott(linear_ramp(2033, 2040, 5, 9))
#plt.show()

In [ ]:
#linear_ramp(2033, 2040, 5, 9).subs({t: 2050})

In [ ]:
import tomli

In [ ]:
with open("levers.toml", "rb") as f:
    levers = tomli.load(f)

In [ ]:
def data_from_levers(ti, levels):
    result = {}
    # XXX don't have to do this every time
    #for symbol, row in constants.iterrows():
    #    result[symbol] = row["Value"]
    for lever, data in levers.items():
        if lever in levels:
            level_data = data["levels"][str(levels[lever])]
            for symbol, symbol_data in level_data["knobs"].items():
                if "value" in symbol_data:
                    y = symbol_data["value"]
                    if isinstance(y, list):
                        result[symbol] = [
                            sy.S(y[i]).evalf(subs={t: ti}) for i in range(len(y))
                        ]
                    else:
                        result[symbol] = sy.S(y).evalf(subs={t: ti})
                elif "values" in symbol_data:
                    assert "years" in symbol_data
                    assert len(symbol_data["years"]) == len(symbol_data["values"]) == 2
                    t1, t2 = symbol_data["years"]
                    y1, y2 = symbol_data["values"]
                    if isinstance(y1, list):
                        result[symbol] = [
                            sy.S(linear_ramp(t1, t2, y1[i], y2[i])).evalf(subs={t: ti})
                            for i in range(len(y1))
                        ]
                    else:
                        result[symbol] = sy.S(linear_ramp(t1, t2, y1, y2)).evalf(subs={t: ti})
                else:
                    raise ValueError("unknown data for symbol %s: %s" % (symbol, symbol_data))
    return result

In [ ]:
import ipywidgets
style = {'description_width': 'initial'}

def lever_widget(lever_id):
    info = levers[lever_id]
    options = [
        (v["description"], k)
        for k, v in info["levels"].items()
    ]
    return ipywidgets.RadioButtons(options=options, description=info["label"], style=style, layout=ipywidgets.Layout(width='50%'))

In [ ]:
values = func({str(x): y for x, y in data_from_levers(2050, {"fertiliser_demand": 1}).items()})

In [ ]:
flows = flows_sym.copy()
flows.value[:] = values
#flows[flows.value > 0]

In [ ]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
w = SankeyWidget(links=flows.to_dict(orient='records'), layout=Layout(width="700", height="300"), margins=dict(left=250, right=50))
w.order = [
    list(sorted(k for k in flows.source.unique() if k.startswith("ProductionOf"))),
    list(sorted(k for k in flows.material.unique())),
    list(sorted(k for k in flows.target.unique() if k.startswith("UseOf"))),
]

In [ ]:
post_funcs = {
    "GHG_fertiliser_prod": sy.lambdify([E_fertiliser_prod, Z_fertiliser], post["GHG_fertiliser_prod"]),
    "GHG_fertiliser_use": sy.lambdify([E_fertiliser_use, Z_fertiliser], post["GHG_fertiliser_use"]),
}

In [ ]:
calc_GHG_fertiliers_y(params_t)[1][0]

In [ ]:
fig = plt.figure(layout=dict(height="300px", width="500px"))
fig.animation_duration = 500
params_t = [
    data_from_levers(ti, {"fertiliser_demand": 1, "fertiliser_use_phase": 1, "fertiliser_production": 1})
    for ti in tvals
]
def calc_GHG_fertiliers_y(params_t):
    y1 = [float(post_funcs["GHG_fertiliser_prod"](p["E_fertiliser_prod"], p["Z_fertiliser"])) for p in params_t]
    y2 = [float(post_funcs["GHG_fertiliser_use"](p["E_fertiliser_use"], p["Z_fertiliser"])) for p in params_t]
    return [
        0*tvals, 
        y1, 
        np.array(y1)+np.array(y2)
    ]
lines = plt.plot(tvals, calc_GHG_fertiliers_y(params_t), fill="between")
plt.ylim(0, 2.1); plt.xlabel("Year"); plt.ylabel("Emissions");

In [ ]:
ipywidgets.HBox([fig, w])

In [ ]:
import ipywidgets
from ipywidgets import interact

@interact(l1=lever_widget("fertiliser_demand"), 
          l2=lever_widget("fertiliser_use_phase"),
          l3=lever_widget("fertiliser_production"),
          ti=(2020., 2050.))
def calc_flows_levels(l1=1, l2=1, l3=1, ti=2050):
    levels = {"fertiliser_demand": l1, "fertiliser_use_phase": l2, "fertiliser_production": l3}
    params = data_from_levers(ti, levels)
    params_t = [
        data_from_levers(ti, levels)
        for ti in tvals
    ]
    flows.value[:] = func({str(x): y for x, y in params.items()})
    w.links = flows.to_dict(orient='records')
    lines.y = calc_GHG_fertiliers_y(params_t)